# Tool Loadout: Optimizing Function Calling with LangChain and Amazon Bedrock

This notebook demonstrates the **Less-is-More** approach from the paper "Less is More: Optimizing Function Calling for LLM Execution on Edge Devices" (arXiv:2411.15399v1).

## Key Concept
Instead of providing all available tools to an LLM at once, we:
1. Ask the LLM to describe what tools it needs (without showing any tools)
2. Use semantic similarity to find the most relevant tools
3. Only provide those selected tools to the LLM for function calling

This reduces confusion, improves accuracy, and decreases execution time and power consumption.

## Setup and Installation

In [ ]:
# Install required packages
!pip install langchain langchain-aws boto3 sentence-transformers scikit-learn numpy

In [ ]:
import boto3
import json
import numpy as np
from typing import List, Dict, Any
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_aws import ChatBedrock
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage

## Configure AWS Bedrock

In [ ]:
# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # Change to your region
)

# Initialize ChatBedrock with Converse API
llm = ChatBedrock(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",  # or another model
    client=bedrock_runtime,
    model_kwargs={
        "temperature": 0.0,
        "max_tokens": 2000
    }
)

print("✓ Bedrock client initialized")

## Define Example Tools

We'll create a diverse set of tools to demonstrate the Less-is-More approach.

In [ ]:
# Define a collection of tools
@tool
def get_weather(location: str) -> str:
    """Get the current weather for a specific location."""
    return f"Weather in {location}: Sunny, 72°F"

@tool
def search_web(query: str) -> str:
    """Search the web for information about a topic."""
    return f"Search results for '{query}': Found 10 relevant articles"

@tool
def calculate_math(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error: Invalid expression"

@tool
def translate_text(text: str, target_language: str) -> str:
    """Translate text to a target language."""
    return f"Translated '{text}' to {target_language}"

@tool
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient."""
    return f"Email sent to {recipient} with subject '{subject}'"

@tool
def get_stock_price(symbol: str) -> str:
    """Get the current stock price for a given symbol."""
    return f"Stock price for {symbol}: $150.25"

@tool
def create_calendar_event(title: str, date: str, time: str) -> str:
    """Create a calendar event with title, date, and time."""
    return f"Calendar event '{title}' created for {date} at {time}"

@tool
def get_news(category: str) -> str:
    """Get latest news for a specific category (e.g., technology, sports, business)."""
    return f"Latest {category} news: 5 articles found"

@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount from one currency to another."""
    return f"Converted {amount} {from_currency} to {to_currency}: {amount * 1.2}"

@tool
def get_directions(origin: str, destination: str) -> str:
    """Get directions from origin to destination."""
    return f"Directions from {origin} to {destination}: 15 miles, 25 minutes"

# Collect all tools
ALL_TOOLS = [
    get_weather,
    search_web,
    calculate_math,
    translate_text,
    send_email,
    get_stock_price,
    create_calendar_event,
    get_news,
    convert_currency,
    get_directions
]

print(f"✓ Defined {len(ALL_TOOLS)} tools")

## Implement Less-is-More Tool Selection

This is the core of the paper's approach.

In [ ]:
class LessIsMoreToolSelector:
    """Implements the Less-is-More approach for dynamic tool selection."""
    
    def __init__(self, all_tools: List[Any], llm: ChatBedrock, top_k: int = 3):
        self.all_tools = all_tools
        self.llm = llm
        self.top_k = top_k
        
        # Initialize embedding model (MPNet as mentioned in paper)
        print("Loading embedding model...")
        self.embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        
        # Create tool descriptions and embeddings (offline step)
        self.tool_descriptions = []
        self.tool_embeddings = []
        
        for tool in all_tools:
            desc = f"{tool.name}: {tool.description}"
            self.tool_descriptions.append(desc)
            embedding = self.embedding_model.encode(desc)
            self.tool_embeddings.append(embedding)
        
        self.tool_embeddings = np.array(self.tool_embeddings)
        print(f"✓ Created embeddings for {len(self.all_tools)} tools")
    
    def recommend_tools(self, user_query: str) -> List[str]:
        """Step 1: Ask LLM to describe ideal tools needed (Tool Recommender)."""
        prompt = f"""Given the following user query, describe what tools or functions would be needed to complete this task.
Do NOT try to answer the query. Instead, list 1-3 tool descriptions that would be helpful.

User query: {user_query}

Provide your response as a JSON list of tool descriptions. Example format:
["A tool to get weather information for a location", "A tool to search for nearby restaurants"]

Tool descriptions:"""
        
        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)
        
        # Parse LLM response
        try:
            # Extract JSON from response
            content = response.content
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0]
            elif "```" in content:
                content = content.split("```")[1].split("```")[0]
            
            recommended_descriptions = json.loads(content.strip())
        except:
            # Fallback: use the entire response as a single description
            recommended_descriptions = [response.content]
        
        return recommended_descriptions
    
    def select_tools(self, user_query: str, recommended_descriptions: List[str]) -> List[Any]:
        """Step 2: Use similarity search to find actual tools (Tool Controller)."""
        # Embed the recommended tool descriptions
        recommended_embeddings = self.embedding_model.encode(recommended_descriptions)
        
        # Calculate similarity scores
        similarities = cosine_similarity(recommended_embeddings, self.tool_embeddings)
        
        # Get top-k most similar tools
        max_similarities = similarities.max(axis=0)
        top_indices = np.argsort(max_similarities)[-self.top_k:][::-1]
        
        selected_tools = [self.all_tools[i] for i in top_indices]
        
        return selected_tools, top_indices
    
    def execute_less_is_more(self, user_query: str) -> Dict[str, Any]:
        """Execute the full Less-is-More pipeline."""
        print(f"\n{'='*60}")
        print(f"User Query: {user_query}")
        print(f"{'='*60}\n")
        
        # Step 1: Tool Recommender
        print("Step 1: Tool Recommender - Asking LLM what tools it needs...")
        recommended_descriptions = self.recommend_tools(user_query)
        print(f"LLM recommended tool descriptions:")
        for i, desc in enumerate(recommended_descriptions, 1):
            print(f"  {i}. {desc}")
        
        # Step 2: Tool Controller
        print(f"\nStep 2: Tool Controller - Finding {self.top_k} most similar tools...")
        selected_tools, indices = self.select_tools(user_query, recommended_descriptions)
        print(f"Selected tools:")
        for i, tool in enumerate(selected_tools, 1):
            print(f"  {i}. {tool.name}: {tool.description}")
        
        return {
            "query": user_query,
            "recommended_descriptions": recommended_descriptions,
            "selected_tools": selected_tools,
            "tool_names": [t.name for t in selected_tools]
        }

print("✓ LessIsMoreToolSelector class defined")

## Initialize the Tool Selector

In [ ]:
# Create the Less-is-More tool selector
selector = LessIsMoreToolSelector(
    all_tools=ALL_TOOLS,
    llm=llm,
    top_k=3  # Select top 3 most relevant tools
)

## Example 1: Weather Query

In [ ]:
result1 = selector.execute_less_is_more(
    "What's the weather like in San Francisco?"
)

## Example 2: Financial Query

In [ ]:
result2 = selector.execute_less_is_more(
    "I need to check the stock price of AAPL and convert 1000 USD to EUR"
)

## Example 3: Complex Multi-Tool Query

In [ ]:
result3 = selector.execute_less_is_more(
    "Search for information about AI and then send an email to my colleague about it"
)

## Comparison: Baseline vs Less-is-More

Let's compare the traditional approach (providing all tools) vs Less-is-More.

In [ ]:
def baseline_approach(user_query: str, all_tools: List[Any]) -> Dict[str, Any]:
    """Traditional approach: provide ALL tools to the LLM."""
    print(f"\n{'='*60}")
    print(f"BASELINE: Providing ALL {len(all_tools)} tools to LLM")
    print(f"{'='*60}\n")
    
    tool_descriptions = "\n".join([
        f"{i+1}. {tool.name}: {tool.description}" 
        for i, tool in enumerate(all_tools)
    ])
    
    prompt = f"""You have access to the following tools:

{tool_descriptions}

User query: {user_query}

Which tools would you use? List the tool names."""
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    print(f"LLM Response:\n{response.content}")
    
    return {
        "query": user_query,
        "num_tools_provided": len(all_tools),
        "response": response.content
    }

# Compare approaches
test_query = "What's the weather in New York?"

print("\n" + "#"*60)
print("# COMPARISON TEST")
print("#"*60)

baseline_result = baseline_approach(test_query, ALL_TOOLS)
less_is_more_result = selector.execute_less_is_more(test_query)

print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"Baseline: Provided {baseline_result['num_tools_provided']} tools")
print(f"Less-is-More: Provided {len(less_is_more_result['selected_tools'])} tools")
print(f"Reduction: {(1 - len(less_is_more_result['selected_tools'])/baseline_result['num_tools_provided'])*100:.1f}%")

## Benefits Analysis

The Less-is-More approach provides:

1. **Reduced Confusion**: LLM sees fewer options, making better decisions
2. **Faster Execution**: Smaller context window = faster processing
3. **Lower Power Consumption**: Less computation required
4. **No Fine-tuning Required**: Works with any LLM out-of-the-box
5. **Scalable**: Can handle large tool sets efficiently

## Advanced: Implementing Search Levels

The paper describes 3 search levels. Let's implement a simple version.

In [ ]:
from sklearn.cluster import KMeans

class MultiLevelToolSelector(LessIsMoreToolSelector):
    """Extended version with multiple search levels."""
    
    def __init__(self, all_tools: List[Any], llm: ChatBedrock, top_k: int = 3, n_clusters: int = 3):
        super().__init__(all_tools, llm, top_k)
        
        # Create tool clusters (Search Level 2)
        print(f"Creating {n_clusters} tool clusters...")
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.tool_clusters = self.kmeans.fit_predict(self.tool_embeddings)
        
        # Group tools by cluster
        self.clustered_tools = {}
        for i, cluster_id in enumerate(self.tool_clusters):
            if cluster_id not in self.clustered_tools:
                self.clustered_tools[cluster_id] = []
            self.clustered_tools[cluster_id].append(self.all_tools[i])
        
        print(f"✓ Created {len(self.clustered_tools)} clusters")
        for cluster_id, tools in self.clustered_tools.items():
            print(f"  Cluster {cluster_id}: {[t.name for t in tools]}")
    
    def select_with_level(self, user_query: str, search_level: int = 1) -> Dict[str, Any]:
        """Select tools using specified search level."""
        print(f"\nUsing Search Level {search_level}")
        
        if search_level == 1:
            # Individual tool selection
            return self.execute_less_is_more(user_query)
        
        elif search_level == 2:
            # Cluster-based selection
            recommended_descriptions = self.recommend_tools(user_query)
            recommended_embeddings = self.embedding_model.encode(recommended_descriptions)
            
            # Find closest cluster
            cluster_centers = self.kmeans.cluster_centers_
            similarities = cosine_similarity(recommended_embeddings, cluster_centers)
            best_cluster = similarities.max(axis=0).argmax()
            
            selected_tools = self.clustered_tools[best_cluster]
            print(f"Selected cluster {best_cluster} with {len(selected_tools)} tools")
            
            return {
                "query": user_query,
                "search_level": 2,
                "cluster_id": int(best_cluster),
                "selected_tools": selected_tools,
                "tool_names": [t.name for t in selected_tools]
            }
        
        else:  # Level 3
            # Use all tools
            print(f"Using all {len(self.all_tools)} tools")
            return {
                "query": user_query,
                "search_level": 3,
                "selected_tools": self.all_tools,
                "tool_names": [t.name for t in self.all_tools]
            }

# Create multi-level selector
multi_selector = MultiLevelToolSelector(
    all_tools=ALL_TOOLS,
    llm=llm,
    top_k=3,
    n_clusters=3
)

## Test Different Search Levels

In [ ]:
test_query = "I need to check stock prices and convert currency"

print("\n" + "#"*60)
print("# TESTING DIFFERENT SEARCH LEVELS")
print("#"*60)

# Test each level
for level in [1, 2, 3]:
    result = multi_selector.select_with_level(test_query, search_level=level)
    print(f"\nLevel {level} selected {len(result['selected_tools'])} tools: {result['tool_names']}")
    print("-" * 60)

## Conclusion

This notebook demonstrates the **Less-is-More** approach for optimizing function calling:

- **Tool Recommender**: LLM describes needed tools without seeing any
- **Tool Controller**: Semantic similarity finds the most relevant actual tools
- **Search Levels**: Different granularities for different query complexities

Key benefits:
- ✓ No fine-tuning required
- ✓ Works with any LLM (we used Amazon Bedrock)
- ✓ Reduces tool confusion
- ✓ Faster execution
- ✓ Lower resource consumption

This approach is particularly valuable for edge deployments where computational resources are limited.